# `multiprocessing`

Parallel computation on a single machine in Python
- one of my most important tools
- Python standard library


## Python standard library parallel computation ecosystem

[Multiprocessing Vs. Threading In Python - Sid Panjwani](https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know/)

`threading` - uses threads (same memory space)
- helps with network issues

`multiprocessing` - uses processes (different memory space)
- help with compute issues

How does this relate to CPU cores
- CPU cores are fixed (usually 4-16 in laptops - depends on your physical hardware)
- more cores = true parallelism (opposed to the very fast task switching done by the OS)
- your computer can have many threads and many processes (depends on the OS)
- the OS will schedule these threads/processes to available cores
- a single thread consumes an entire core

[Multithreading and multicore differences](https://stackoverflow.com/questions/11835046/multithreading-and-multicore-differences)

*But my CPU core has two threads*
- this is a different use of the term (the hardware thread)
- CPU having threads allows a core to run thread in parallel, as if there were multiple cores
- known as hyperthreading


## Why do we need `multiprocessing`?

Python has a Global Interpreter Lock (GIL) that prevents parallelizing computation across multiple cores
- Python is not thread safe
- requires a lock when accessing an object (a form of memory management)


## What can be hard in multiprocessing?

Sharing things between processes
- solution = don't use it in this way
- make every process independent
- a functional style = no interaction (because interaction = side effects!)


## `multiprocessing` 101

We map functions to data
- but in parallel!

First let's do a simple `map` in Python:

In [1]:
import time
import numpy as np

def subtract(x, sleep=0.01):
    time.sleep(sleep)
    return x*x

data = np.random.uniform(0, 100, size=100).tolist()
st = time.time()

result = list(map(subtract, data))
print(time.time() - st)

1.173375129699707


Let's parallelize this using `multiprocessing`:

In [2]:
from multiprocessing import Pool

num_process = 8
st = time.time()

with Pool(num_process) as pool:
    out = pool.map(subtract, data)
    
print(time.time() - st)

0.24825763702392578


A common use case is to have arguments for the function being mapped:

In [4]:
from functools import partial

st = time.time()
with Pool(num_process) as p:
    rewards = p.map(partial(subtract, sleep=0.1), data)
    
print(time.time() - st)

1.675908088684082


Note that when we remove our sleep, the non-mulitprocessing `map` is faster:

In [5]:
st = time.time()
result = list(map(partial(subtract, sleep=0.0), data))
print(time.time() - st)

0.0006787776947021484


Distributed computation has overhead (fixed + variable) 
- make sure your function runs long enough to justify it

## Exercise - bitcoin mining

Write multiprocessed code to solve a hashing problem (similar to how *proof of work* works in Bitcoin)
- take a given input string (base string)
- add strings on the end of it until you get a hash with a leading `0`

We can hash in Python:

In [6]:
from zlib import adler32
str(adler32('I miss DSR already'.encode()))

'928581169'

We can add characters onto the end of this string and we will get a different hash:

In [7]:
str(adler32('I miss DSR already!'.encode()))

'1034618450'

There are a few ways to solve this - one is to write a function that:
- randomly selects n characters
- adds those characters onto the string
- if the hash starts with 1, return the characters & a success code
- otherwise return the characters and a failure code

This function can be run in parallel :)

1. write the function
2. normal `map`
3. multiprocess :)

In [19]:
import string

base_string = 'I miss DSR already'
alpha = string.ascii_lowercase

def check_string(stri):
    ha = str(adler32(stri.encode()))
    
base = 'I miss DSR already'
for _ in range(100):
    import random
    new = random.choice(alpha)
    check_string(base + new)

In [35]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()
from sklearn.datasets import load_iris

data = load_iris()['data']
import numpy as np

ss
x = np.random.normal(0, 1, size=size)
y = data[:, -1]

tree.fit(x, y)
pred = tree.predict(x)
pred

array([0.25, 0.2 , 0.2 , 0.2 , 0.2 , 0.4 , 0.3 , 0.2 , 0.2 , 0.15, 0.2 ,
       0.2 , 0.2 , 0.1 , 0.2 , 0.4 , 0.4 , 0.25, 0.3 , 0.3 , 0.2 , 0.4 ,
       0.2 , 0.5 , 0.2 , 0.2 , 0.4 , 0.2 , 0.2 , 0.2 , 0.2 , 0.4 , 0.1 ,
       0.2 , 0.15, 0.2 , 0.2 , 0.1 , 0.2 , 0.2 , 0.3 , 0.3 , 0.2 , 0.6 ,
       0.4 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 1.4 , 1.5 , 1.5 , 1.3 , 1.5 ,
       1.3 , 1.6 , 1.  , 1.3 , 1.4 , 1.  , 1.5 , 1.  , 1.4 , 1.3 , 1.4 ,
       1.5 , 1.  , 1.5 , 1.1 , 1.8 , 1.3 , 1.5 , 1.2 , 1.3 , 1.4 , 1.4 ,
       1.7 , 1.5 , 1.  , 1.1 , 1.  , 1.2 , 1.6 , 1.5 , 1.6 , 1.5 , 1.3 ,
       1.3 , 1.3 , 1.2 , 1.4 , 1.2 , 1.  , 1.3 , 1.2 , 1.3 , 1.3 , 1.1 ,
       1.3 , 2.5 , 1.9 , 2.1 , 1.8 , 2.2 , 2.1 , 1.7 , 1.8 , 1.8 , 2.5 ,
       2.  , 1.9 , 2.1 , 2.  , 2.4 , 2.3 , 1.8 , 2.2 , 2.3 , 1.5 , 2.3 ,
       2.  , 2.  , 1.8 , 2.3 , 1.8 , 1.8 , 1.8 , 2.15, 1.6 , 1.9 , 2.  ,
       2.15, 1.5 , 1.4 , 2.3 , 2.4 , 1.8 , 1.8 , 2.1 , 2.4 , 2.3 , 1.9 ,
       2.3 , 2.3 , 2.3 , 1.9 , 2.  , 2.3 , 1.8 ])